## Adobe TID Classification
The goal is to assign the unassigned marketing campaigns (TIDs) from Adobe Analytics to their corresponding Managers for both Lex and CR. To do this, we need to use the map from `TID` to `Manager` found in the marketing databases.

The SQL programs used to query the databases are `Adobe Classification Query (CR).sql` and `Adobe Classification Query (Lex).sql`. The output from these queries is stored in the CSV files `CR TID Map.csv` and `Lex TID Map.csv`. In what follows, we read these results in and use them to assign the unknown TIDs to a Manager. We then store the classified TIDs as a template and upload them into Adobe Analytics.

In [1]:
## Import packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid')

## Step 1 - Download the unassigned TIDs from Adobe Analytics

- Login to Adobe Analytics
- Select AID Classification from the Workspace
- Drag AID v1 onto Unspecified
- Add the brand at the top as a segment (Lexington Law or Credit Repair) by searching components for `Report Suite (v37)` and selecting either `Lexington Law` or `Credit Repair`
- Expand the Unspecified AID column:

<img src="img/Adobe_Analytics_Screenshot.png" alt="Drawing" style="width: 450px;"/>

- Download CSV for each brand
    - Project -> Download CSV (for each brand)

## Step 2 - Generate the map between TID and Department using SQL

#### Here is the SQL query for CR:

#### And here is the SQL query for Lex:

## Step 3 - Assign the TIDs to a Manager

In [39]:
## Read in unassigned TIDs for LEX and CR
# CR_unassigned_TIDs = pd.read_csv('CR Unassigned TIDs.csv',header=13,names=['AID (v1)','Counts'])
# LEX_unassigned_TIDs = pd.read_csv('LEX Unassigned TIDs.csv',header=13,names=['AID (v1)','Counts'])
LEX_unassigned_TIDs = pd.read_csv('LEX_undefined_TIDs.csv')
CR_unassigned_TIDs = pd.read_csv('CR_undefined_TIDs.csv')


## Convert these DataFrames to lists
CR_unassigned_TIDs = CR_unassigned_TIDs['AID (v1)'].astype(str).tolist()
LEX_unassigned_TIDs = LEX_unassigned_TIDs['AID (v1)'].astype(str).tolist()

## Read in the tbl_campaigns map generated by the SQL query (e.g., TID and Manager) 
CR_SQL_results = pd.read_csv('CR TID Map New.csv')
LEX_SQL_results = pd.read_csv('LEX TID Map New.csv')

## Make sure dtype will match
LEX_SQL_results['Key'] = LEX_SQL_results['Key'].astype(str)
CR_SQL_results['Key'] = CR_SQL_results['Key'].astype(str)

## Assign missing TIDs to a manager using the SQL results
CR_assigned_TIDs = CR_SQL_results.loc[CR_SQL_results['Key'].isin(CR_unassigned_TIDs)]
LEX_assigned_TIDs = LEX_SQL_results.loc[LEX_SQL_results['Key'].isin(LEX_unassigned_TIDs)]

## Add a warning if unassigned TIDs aren't assigned
counter = 0
for TID in LEX_unassigned_TIDs:
    if TID not in LEX_assigned_TIDs['Key'].to_list():
        print("\033[1;31;47m WARNING: TID ",TID,"NOT ASSIGNED  \n")   
        counter += 1
        
print("COVERAGE = ",1-counter/len(LEX_unassigned_TIDs))

LEX_assigned_TIDs


















































COVERAGE =  0.9423529411764706


,Key,Name,Manager,Department,Brand,Type,Channel,Network,Delivery,Device
0,1,Anonymous,Brand - Unassigned,Brand - Unassigned,LexingtonLaw,Other,Other,Other,NaN,NaN
41,50,Google: Brand Desktop - LEX 1,Online - PPC Google BR (New),Paid Search - Brand,LexingtonLaw,Other,Other,Google,NaN,NaN
43,52,newcheckingaccount.com,Lead Gen - Short Form,Lead Gen,LexingtonLaw,Other,Other,Other,NaN,NaN
114,123,NewImpulse2 test [NA],Lead Gen - Short Form,Lead Gen,LexingtonLaw,Other,Other,Other,NaN,NaN
133,142,Commission Junction (nopop),Online - Affiliate NB,Affiliate,LexingtonLaw,Other,Other,Other,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
9633,34641,Remarketing- New 4,Remarketing - Email,Remarketing,LexingtonLaw,Other,Other,Other,NaN,NaN
9634,34642,Remarketing- New 5,Remarketing - Email,Remarketing,LexingtonLaw,Other,Other,Other,NaN,NaN
9635,34643,Remarketing- New 6,Remarketing - Email,Remarketing,LexingtonLaw,Other,Other,Other,NaN,NaN
9636,34644,Remarketing- New 7,Remarketing - Email,Remarketing,LexingtonLaw,Other,Other,Other,NaN,NaN


## Step 4 - Output Results as Tab-Deliminated Text File

In [40]:
## First read in an old template to get the header syntax
temp_file = open("sample_template.txt")
number_of_lines = 4
headertext = ""
for i in range(number_of_lines):
    headertext += temp_file.readline()
headertext = headertext.rstrip() ## remove the last newline character (formatting

## Now export the assigned TIDs as formatted templates
np.savetxt('CR_assigned_TIDS.txt', CR_assigned_TIDs.values, fmt='%s', delimiter='\t', header=headertext, comments='')
np.savetxt('LEX_assigned_TIDS.txt', LEX_assigned_TIDs.values, fmt='%s', delimiter='\t', header=headertext, comments='')

print("Done")

Done


## Step 5 - Upload to Adobe
Go to `Admin` then `Classification Importer`. Click `Import File` and select `AID (v1)` for data set to be classified.

<img src="img/classification_importer.png" alt="Drawing" style="width: 450px;"/>

#### A successful import should look something like this:


<img src="img/import_success.png" alt="Drawing" style="width: 350px;"/>

## Summary

To summarize, we began by reading in the unassigned marketing campaigns that we downloaded from Adobe Analytics (Step 1). We then used `redacted` in the marketing database to download the metadata for all TIDs (Step 2). Next, we used this metadata to map each unassigned campaign to a manager (Step 3). Last, we output the results to a specially formatted file (Step 4) and imported this file to Adobe Analytics (Step 5).